In [ ]:
import numpy as np
import pandas as pd
import glob
import os
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from pathlib import Path

In [ ]:
def get_ground_truth(args):
    (collectionName, phoneName), df = args
    
    path = root_dir / f"train/{collectionName}/{phoneName}/ground_truth.csv"
    target_df = pd.read_csv(path)
    output_df = pd.DataFrame()
    # merge derived and target by 'millisSinceGpsEpoch'
    for epoch, epoch_df in df.groupby('millisSinceGpsEpoch'):
        idx = (target_df['millisSinceGpsEpoch'] - epoch).abs().argmin()
        epoch_diff = epoch - target_df.loc[idx, 'millisSinceGpsEpoch']
        epoch_df['epoch_diff'] = epoch_diff
        epoch_df['target_latDeg'] = target_df.loc[idx, 'latDeg']
        epoch_df['target_lngDeg'] = target_df.loc[idx, 'lngDeg']
        output_df = pd.concat([output_df, epoch_df]).reset_index(drop=True)    
    return output_df

In [ ]:
import plotly.express as px

def visualize_trafic(df):
    fig = px.scatter_mapbox(df,
                            
                            # Here, plotly gets, (x,y) coordinates
                            lat="latDeg",
                            lon="lngDeg",
                            text='phoneName',
                            
                            #Here, plotly detects color of series
                            color="collectionName",
                            labels="collectionName",
                            
                            zoom=9,
                            center={"lat":37.423576, "lon":-122.094132},
                            height=600,
                            width=800)
    fig.update_layout(mapbox_style='stamen-terrain')
    fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
    fig.update_layout(title_text="GPS trafic")
    fig.show()


def visualize_collection(df, collection_name='2021-04-29-US-SJC-2'):
    gt_df = pd.read_csv('../../input/all_ground_truth.csv')
    gt_df['phoneName'] = 'ground truth'
    gt_df = pd.concat([df, gt_df])
    fig = px.scatter_mapbox(gt_df[gt_df['collectionName']==collection_name],
                                
                            # Here, plotly gets, (x,y) coordinates
                            lat="latDeg",
                            lon="lngDeg",
                                
                            #Here, plotly detects color of series
                            color="phoneName",
                            labels="phoneName",
                                
                            zoom=9,
                            center={"lat":37.423576, "lon":-122.094132},
                            height=600,
                            width=800)
    fig.update_layout(mapbox_style='stamen-terrain')
    fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
    fig.update_layout(title_text="GPS trafic")
    fig.show()

In [ ]:
# metric
def calc_haversine(lat1, lon1, lat2, lon2):
    """Calculates the great circle distance between two points
    on the earth. Inputs are array-like and specified in decimal degrees.
    """
    RADIUS = 6_367_000
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + \
        np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    dist = 2 * RADIUS * np.arcsin(a**0.5)
    return dist

In [ ]:
import simdkalman

# define kf model
T = 1.0
state_transition = np.array([[1, 0, T, 0, 0.5 * T ** 2, 0], [0, 1, 0, T, 0, 0.5 * T ** 2], [0, 0, 1, 0, T, 0],
                             [0, 0, 0, 1, 0, T], [0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 1]])
process_noise = np.diag([1e-5, 1e-5, 5e-6, 5e-6, 1e-6, 1e-6]) + np.ones((6, 6)) * 1e-9
observation_model = np.array([[1, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0]])
observation_noise = np.diag([5e-5, 5e-5]) + np.ones((2, 2)) * 1e-9

kf = simdkalman.KalmanFilter(
        state_transition = state_transition,
        process_noise = process_noise,
        observation_model = observation_model,
        observation_noise = observation_noise)

In [ ]:
def apply_kf_smoothing(df, kf_=kf):
    unique_paths = df[['collectionName', 'phoneName']].drop_duplicates().to_numpy()
    for collection, phone in tqdm(unique_paths):
        cond = np.logical_and(df['collectionName'] == collection, df['phoneName'] == phone)
        data = df[cond][['latDeg', 'lngDeg']].to_numpy()
        data = data.reshape(1, len(data), 2)
        smoothed = kf_.smooth(data)
        df.loc[cond, 'latDeg'] = smoothed.states.mean[0, :, 0]
        df.loc[cond, 'lngDeg'] = smoothed.states.mean[0, :, 1]
    return df

In [ ]:
def linear_interpolation(input_df):
    dfs = pd.DataFrame()
    for (collectionName, phoneName), df in input_df.groupby(['collectionName','phoneName']):

        df['delta'] = calc_haversine(df['latDeg'], df['lngDeg'], df['latDeg'].shift(1), df['lngDeg'].shift(1))
        df['time_delta'] = df['millisSinceGpsEpoch'] - df['millisSinceGpsEpoch'].shift(1)
        df['delta'].fillna(0, inplace=True)
        df['time_delta'].fillna(0, inplace=True)
        df['speed'] = df['delta'] / (df['time_delta']/1000)  # m/s
        df['speed'].fillna(0, inplace=True)

        # 一度欠損値にする
        df.loc[50<df['speed'], ['latDeg', 'lngDeg']] = np.nan
        df['dummy_datetime'] = pd.to_datetime(df['millisSinceGpsEpoch'])
        df = df.set_index('dummy_datetime')

        # 時間に合わせて線形補間
        df = df.interpolate(method='time').reset_index(drop=True)
        dfs = pd.concat([dfs, df]).reset_index(drop=True)
    return dfs

In [ ]:
# config
EXP_NAME = str(Path().resolve()).split('/')[-1]

In [ ]:
EXP_NAME

In [ ]:
# waypointを補正したdataset
root_dir = Path('../../input/')
test_df = pd.read_csv(root_dir / "baseline_locations_test.csv")
sub_df = pd.read_csv(root_dir / 'sample_submission.csv')

## LI -> KF  

In [ ]:
import multiprocessing
train_df = pd.read_csv(root_dir / "baseline_locations_train.csv")
processes = multiprocessing.cpu_count()
with multiprocessing.Pool(processes=processes) as pool:
    gr = train_df.groupby(['collectionName','phoneName'])
    dfs = pool.imap_unordered(get_ground_truth, gr)
    dfs = tqdm(dfs, total=len(gr))
    dfs = list(dfs)
train_df = pd.concat(dfs).sort_values(['collectionName', 'phoneName', 'millisSinceGpsEpoch'])
calc_haversine(train_df['latDeg'],train_df['lngDeg'],train_df['target_latDeg'], train_df['target_lngDeg']).mean()

In [ ]:
visualize_collection(train_df)

In [ ]:
train_df = linear_interpolation(train_df)
calc_haversine(train_df['latDeg'],train_df['lngDeg'],train_df['target_latDeg'], train_df['target_lngDeg']).mean()

In [ ]:
visualize_collection(train_df)

In [ ]:
def get_stopping_group(a, n=5):
    rle = [[a[0], 1]]

    for elem in a[1: ]:
        if rle[-1][0] == elem:
            rle[-1][1] += 1
        else:
            rle.append([elem, 1])
    id = 1
    result = []
    for elem, cnt in rle:
        if elem == 1 and cnt >= n:
            result += [id] * cnt
            id += 1
        else:
            result += [0] * cnt
    return result


def apply_stopping_pp(input_df, speed_thr=1, cont_thr=5):
    # 速度が1m/s以下のものは停止しているとみなしてその近い時間で平均を取る
    input_df['is_stopping'] = 0
    input_df.loc[train_df['speed']<speed_thr, 'is_stopping'] = 1

    coll_df_list = []
    for collection_name, coll_df in input_df.groupby('collectionName'):
        a = coll_df.sort_values('millisSinceGpsEpoch')['is_stopping'].to_numpy()
        coll_df["stopping_group"] = get_stopping_group(a, n=cont_thr)
        stopping_point_df = coll_df[coll_df["stopping_group"]!=0].groupby("stopping_group")[["latDeg", "lngDeg"]].mean().reset_index()

        for i, row in stopping_point_df.iterrows():
            coll_df.loc[coll_df["stopping_group"]==row["stopping_group"], "latDeg"] = row["latDeg"]
            coll_df.loc[coll_df["stopping_group"]==row["stopping_group"], "lngDeg"] = row["lngDeg"]
        coll_df_list.append(coll_df)

    output_df = pd.concat(coll_df_list).reset_index(drop=True)
    return output_df

In [ ]:
_train_df = apply_stopping_pp(train_df, speed_thr=1, cont_thr=10)
calc_haversine(_train_df['latDeg'],_train_df['lngDeg'],_train_df['target_latDeg'], _train_df['target_lngDeg']).mean()

In [ ]:
visualize_collection(_train_df)

In [ ]:
train_df = apply_kf_smoothing(train_df)
calc_haversine(train_df['latDeg'],train_df['lngDeg'],train_df['target_latDeg'], train_df['target_lngDeg']).mean()

In [ ]:
visualize_collection(train_df)

In [ ]:
# 同じcollection、epochである端末の予測値を平均する(アンサンブル的な感じ)
train_df = train_df.merge(train_df.groupby(['collectionName', 'millisSinceGpsEpoch'])[['latDeg', 'lngDeg']].mean(), on=['collectionName', 'millisSinceGpsEpoch'])
train_df = train_df.drop(['latDeg_x', 'lngDeg_x'], axis=1).rename(columns={'latDeg_y':'latDeg', 'lngDeg_y':'lngDeg'})

calc_haversine(train_df['latDeg'],train_df['lngDeg'],train_df['target_latDeg'], train_df['target_lngDeg']).mean()

In [ ]:
train_df = linear_interpolation(train_df)
calc_haversine(train_df['latDeg'],train_df['lngDeg'],train_df['target_latDeg'], train_df['target_lngDeg']).mean()

## test

In [ ]:
import pickle
def to_pickle(filename, obj):
    with open(filename, mode='wb') as f:
        pickle.dump(obj, f)
        
def from_pickle(filename):
    with open(filename, mode='rb') as f:
        obj = pickle.load(f)
    return obj

In [ ]:
test_df = linear_interpolation(test_df)

In [ ]:
test_df.info()

In [ ]:
test_df = apply_kf_smoothing(test_df)

In [ ]:
# 同じcollection、epochである端末の予測値を平均する(アンサンブル的な感じ)
test_df = test_df.merge(test_df.groupby(['collectionName', 'millisSinceGpsEpoch'])[['latDeg', 'lngDeg']].mean(), on=['collectionName', 'millisSinceGpsEpoch'])
test_df = test_df.drop(['latDeg_x', 'lngDeg_x'], axis=1).rename(columns={'latDeg_y':'latDeg', 'lngDeg_y':'lngDeg'})

In [ ]:
test_df = apply_stopping_pp(test_df)

In [ ]:
# sub_df = sub_df.assign(
#     latDeg = test_df['latDeg'],
#     lngDeg = test_df['lngDeg']
# )
sub_df = sub_df.drop(['latDeg', 'lngDeg'], axis=1).merge(test_df[['phone', 'millisSinceGpsEpoch', 'latDeg', 'lngDeg']], on=['phone', 'millisSinceGpsEpoch'])
sub_df.to_csv('submission.csv', index=False)

In [ ]:
_sub_df = pd.read_csv('../exp002/submission.csv')

In [ ]:
calc_haversine(sub_df['latDeg'],sub_df['lngDeg'],
               _sub_df['latDeg'], _sub_df['lngDeg']).mean()